In [ ]:
# DGHK23，嵌入不等式hint， <v,s> <= l
cd framework

In [ ]:
load("../framework/LWE.sage")
import numpy as np
import random

In [ ]:
n = 128
m = n
q = 3329
D_s = build_centered_binomial_law(3)
D_e = D_s
lwe_instance = LWE(n, q, m, D_e, D_s)
ebdd = lwe_instance.embed_into_EBDD()
beta, delta = ebdd.estimate_attack()

In [ ]:
# 使用拒绝采样生成[-3*sigma,3*sigma]之间满足高斯分布的随机数
def sample_discrete_gaussian(sigma):
    while True:
        x = random.randint(int(-3 * sigma), int(3 * sigma)) 
        acceptance_prob = math.exp(-x**2 / (2 * sigma**2)) 
        if random.uniform(0, 1) < acceptance_prob: 
            return x
        
# 生成不等式hint，<v,s>=l-noisy，<v,s><l
def generate_se_eta_sigma_ineq_hint(m, n, q, sigma, k):
  V = []
  L = []

  for i in range(k):
    D_e = {-3: 1/64, -2: 6/64, -1: 15/64, 0: 20/64, 1: 15/64, 2: 6/64, 3: 1/64}
    values, probabilities = zip(*D_e.items())
    v = np.array(np.random.choice(values, size=m+n, p=probabilities))
    v_vec = vec(v)
    noisy = sample_discrete_gaussian(sigma)+ 3 * sigma
    # print(noisy)
    l = ebdd.leak(v_vec) + noisy
    V.append(v)
    L.append(l)
  print("L",L)
  return V,L

In [ ]:
nph_Kyber128 = [0, 40, 80, 120, 160, 200, 240, 280, 320, 360, 400, 440, 480, 520, 560, 600, 640, 680, 720, 760, 800]
dis_Kyber128_our = [18.52, 18.52, 18.52, 18.52, 17.89, 17.09, 16.82, 15.94, 15.07, 14.03, 14.35, 12, 11.58, 10.34, 9.8, 6.56, 4.24, 3.61, 0, 0, 0]
nph_Kyber256 = []
dis_Kyber256_our = []
nph_Kyber512 = []
dis_Kyber512_our = []

num_hint = 800
sigma = 2
V, L = generate_se_eta_sigma_ineq_hint(m, n, q, sigma, num_hint)
BETA = []
index = 0
for j in range(num_hint):
    if j == nph_Kyber128[index]:
        index += 1
        st = dis_Kyber128_our[index]/dis_Kyber128_our[0]
        beta, delta = ebdd.estimate_attack_SMY(st)
        print("beta: ", beta)
        BETA.append(beta)
    print("the ",j+1,"-th secret error ineq hint")
    _ = ebdd.integrate_ineq_hint(vec(V[j]), L[j])
print(BETA)